# Temporal CNN in Keras and TensorFlow

Traditionally, we use Convolutional Neural Networks (CNNs) for image classification problems and Long Short Term Memory (LSTM) networks for time series.  However, recent research has shown CNN's to be beneficial at time series problems. We begin similarly to how we started for LSTM by attempting to classify simple numeric sequences.  The data preparation is nearly identical to LSTM; however, the neural network consists of a **Conv1D** layer this time around.  The following code prepares the data and constructs the neural network.

In [2]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D
import numpy as np

max_features = 4 # 0,1,2,3 (total of 4)
x = [
    [[0],[1],[1],[0],[0],[0]],
    [[0],[0],[0],[2],[2],[0]],
    [[0],[0],[0],[0],[3],[3]],
    [[0],[2],[2],[0],[0],[0]],
    [[0],[0],[3],[3],[0],[0]],
    [[0],[0],[0],[0],[1],[1]]
]
x = np.array(x,dtype=np.float32)
y = np.array([1,2,3,2,3,1],dtype=np.int32)

# Convert y2 to dummy variables
y2 = np.zeros((y.shape[0], max_features),dtype=np.float32)
y2[np.arange(y.shape[0]), y] = 1.0
y2 = np.asarray(y2).astype('float32').reshape((-1,1,4))
print(y2)

print('Build model...')
model = Sequential()
model.add(Conv1D(128, kernel_size=x.shape[1], input_shape=(None, 1)))
model.add(Dense(4, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x,y2,epochs=200)
pred = model.predict(x)
predict_classes = np.argmax(pred,axis=1)
print("Predicted classes: {}",predict_classes)
print("Expected classes: {}",predict_classes)

[[[0. 1. 0. 0.]]

 [[0. 0. 1. 0.]]

 [[0. 0. 0. 1.]]

 [[0. 0. 1. 0.]]

 [[0. 0. 0. 1.]]

 [[0. 1. 0. 0.]]]
Build model...
Train...
Train on 6 samples
Epoch 1/200
6/6 [==============================] - 1s 240ms/sample - loss: 0.7315 - accuracy: 0.3333
Epoch 2/200
6/6 [==============================] - 0s 583us/sample - loss: 0.7166 - accuracy: 0.4167
Epoch 3/200
6/6 [==============================] - 0s 583us/sample - loss: 0.7021 - accuracy: 0.5833
Epoch 4/200
6/6 [==============================] - 0s 584us/sample - loss: 0.6882 - accuracy: 0.5833
Epoch 5/200
6/6 [==============================] - 0s 583us/sample - loss: 0.6746 - accuracy: 0.6667
Epoch 6/200
6/6 [==============================] - 0s 583us/sample - loss: 0.6616 - accuracy: 0.7917
Epoch 7/200
6/6 [==============================] - 0s 500us/sample - loss: 0.6489 - accuracy: 0.7917
Epoch 8/200
6/6 [==============================] - 0s 500us/sample - loss: 0.6367 - accuracy: 0.7500
Epoch 9/200
6/6 [========================

Now that the model is trained, we will run it with sample input and see that it classifies correctly.

In [3]:
def runit(model, inp):
    inp = np.array(inp,dtype=np.float32)
    pred = model.predict(inp)
    return np.argmax(pred[0])

print( runit( model, [[[0],[1],[1],[0],[0],[0]]] ))


1


### Sun Spots Example - CNN

We now look at an example of CNN regression to predict sunspots.  You can find the data files needed for this example at the following location.

* [Sunspot Data Files](http://www.sidc.be/silso/datafiles#total)
* [Download Daily Sunspots](http://www.sidc.be/silso/INFO/sndtotcsv.php) - 1/1/1818 to now.

We use the following code to load the sunspot file:


In [4]:
import pandas as pd
import os

# Replacce the following path with your own file. It can be downloaded from:
# http://www.sidc.be/silso/INFO/sndtotcsv.php
path = "./data/"
    
filename = os.path.join(path,"SN_d_tot_V2.0.csv")    
names = ['year', 'month', 'day', 'dec_year', 'sn_value' , 
         'sn_error', 'obs_num']
df = pd.read_csv(filename,sep=';',header=None,names=names,
                 na_values=['-1'], index_col=False)

print("Starting file:")
print(df[0:10])

print("Ending file:")
print(df[-10:])

FileNotFoundError: [Errno 2] File ./data/SN_d_tot_V2.0.csv does not exist: './data/SN_d_tot_V2.0.csv'

As you can see, there is quite a bit of missing data near the end of the file.  We want to find the starting index where the missing data no longer occurs.  This approach is somewhat sloppy; it would be better to find a use for the data between missing values.  However, the point of this example is to show how to use CNN with a somewhat simple time-series.

In [ ]:
start_id = max(df[df['obs_num'] == 0].index.tolist())+1  # Find the last zero and move one beyond
print(start_id)
df = df[start_id:] # Trim the rows that have missing observations

Next, we break the data into test and training sets.  We will train the neural network in the years before 2000 and test it with years after.

In [ ]:
df['sn_value'] = df['sn_value'].astype(float)
df_train = df[df['year']<2000]
df_test = df[df['year']>=2000]

spots_train = df_train['sn_value'].tolist()
spots_test = df_test['sn_value'].tolist()

print("Training set has {} observations.".format(len(spots_train)))
print("Test set has {} observations.".format(len(spots_test)))

We will make use of the same **to_sequences** function previously used in this course.  This function allows us to convert the training and test data into the 2D sequences that will train the neural network. 

In [ ]:
import numpy as np

def to_sequences(seq_size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE-1):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)
        
    return np.array(x),np.array(y)
    
    
SEQUENCE_SIZE = 25
x_train,y_train = to_sequences(SEQUENCE_SIZE,spots_train)
x_test,y_test = to_sequences(SEQUENCE_SIZE,spots_test)

print("Shape of training set: {}".format(x_train.shape))
print("Shape of test set: {}".format(x_test.shape))

We can display the training data, which is essentially a 2D matrix where each row is a sequence of the same length.

In [ ]:
x_train

We are now ready to build and train the neural network.  This code is similar to the previous classification example; however, now we are performing regression.  

In [ ]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, 
from tensorflow.keras.layers import Dropout, MaxPooling1D, Flatten
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

print('Build model...')
model = Sequential()
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', 
                 input_shape=(SEQUENCE_SIZE,1)))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50, 
                        verbose=1, mode='auto', restore_best_weights=True)
print('Train...')

model.fit(x_train,y_train,validation_data=(x_test,y_test),
          callbacks=[monitor],verbose=2,epochs=1000)

Finally, we evaluate the accuracy of the predictions.

In [ ]:
from sklearn import metrics

# from sklearn import metrics
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred.flatten(),y_test))
print("Score (RMSE): {}".format(score))